In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from PIL import Image

import torch
from torchvision import transforms, datasets
import numpy as np
import os
import numpy as np
import torch
from torch.autograd import Variable
from src.get_nets import PNet, RNet, ONet
from src.box_utils import nms, calibrate_box, get_image_boxes, convert_to_square
from src.first_stage import run_first_stage
import torch.nn as nn

In [3]:
pnet = PNet()
rnet = RNet()
onet = ONet()
onet.eval()

ONet(
  (features): Sequential(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (prelu1): PReLU(num_parameters=32)
    (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (prelu2): PReLU(num_parameters=64)
    (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (prelu3): PReLU(num_parameters=64)
    (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv4): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1))
    (prelu4): PReLU(num_parameters=128)
    (flatten): Flatten()
    (conv5): Linear(in_features=1152, out_features=256, bias=True)
    (drop5): Dropout(p=0.25)
    (prelu5): PReLU(num_parameters=256)
  )
  (conv6_1): Linear(in_features=256, out_features=2, bias=True)
  (conv6_2): Linear(in_features=256, out_features=4, bias=True)
  (co

In [4]:
class OnetFeatures(nn.Module):
    def __init__(self, original_model):
        super(OnetFeatures, self).__init__()
        self.features = nn.Sequential(*list(onet.children())[:-3])
        
    def forward(self, x):
        x = self.features(x)
        return x

def get_face_features(image, min_face_size=20.0,
                 thresholds=[0.6, 0.7, 0.8],
                 nms_thresholds=[0.7, 0.7, 0.7]):
    """
    Arguments:
        image: an instance of PIL.Image.
        min_face_size: a float number.
        thresholds: a list of length 3.
        nms_thresholds: a list of length 3.

    Returns:
        two float numpy arrays of shapes [n_boxes, 4] and [n_boxes, 10],
        bounding boxes and facial landmarks.
    """

    # LOAD MODELS
    pnet = PNet()
    rnet = RNet()
    onet = ONet()
    onet.eval()

    # BUILD AN IMAGE PYRAMID
    width, height = image.size
    min_length = min(height, width)

    min_detection_size = 12
    factor = 0.707  # sqrt(0.5)

    # scales for scaling the image
    scales = []

    # scales the image so that
    # minimum size that we can detect equals to
    # minimum face size that we want to detect
    m = min_detection_size/min_face_size
    min_length *= m

    factor_count = 0
    while min_length > min_detection_size:
        scales.append(m*factor**factor_count)
        min_length *= factor
        factor_count += 1

    # STAGE 1

    # it will be returned
    bounding_boxes = []

    # run P-Net on different scales
    for s in scales:
        boxes = run_first_stage(image, pnet, scale=s, threshold=thresholds[0])
        bounding_boxes.append(boxes)

    # collect boxes (and offsets, and scores) from different scales
    bounding_boxes = [i for i in bounding_boxes if i is not None]
    bounding_boxes = np.vstack(bounding_boxes)

    keep = nms(bounding_boxes[:, 0:5], nms_thresholds[0])
    bounding_boxes = bounding_boxes[keep]

    # use offsets predicted by pnet to transform bounding boxes
    bounding_boxes = calibrate_box(bounding_boxes[:, 0:5], bounding_boxes[:, 5:])
    # shape [n_boxes, 5]

    bounding_boxes = convert_to_square(bounding_boxes)
    bounding_boxes[:, 0:4] = np.round(bounding_boxes[:, 0:4])

    # STAGE 2

    img_boxes = get_image_boxes(bounding_boxes, image, size=24)
    img_boxes = Variable(torch.FloatTensor(img_boxes), volatile=True)
    output = rnet(img_boxes)
    offsets = output[0].data.numpy()  # shape [n_boxes, 4]
    probs = output[1].data.numpy()  # shape [n_boxes, 2]

    keep = np.where(probs[:, 1] > thresholds[1])[0]
    bounding_boxes = bounding_boxes[keep]
    bounding_boxes[:, 4] = probs[keep, 1].reshape((-1,))
    offsets = offsets[keep]

    keep = nms(bounding_boxes, nms_thresholds[1])
    bounding_boxes = bounding_boxes[keep]
    bounding_boxes = calibrate_box(bounding_boxes, offsets[keep])
    bounding_boxes = convert_to_square(bounding_boxes)
    bounding_boxes[:, 0:4] = np.round(bounding_boxes[:, 0:4])

    # STAGE 3

    img_boxes = get_image_boxes(bounding_boxes, image, size=48)
    if len(img_boxes) == 0: 
        return [], []
    img_boxes = Variable(torch.FloatTensor(img_boxes), volatile=True)
    output = onet(img_boxes)
    
    faceFeatureModel = OnetFeatures(onet)

    featureOutputs = faceFeatureModel(img_boxes)
    
    landmarks = output[0].data.numpy()  # shape [n_boxes, 10]
    offsets = output[1].data.numpy()  # shape [n_boxes, 4]
    probs = output[2].data.numpy()  # shape [n_boxes, 2]

    keep = np.where(probs[:, 1] > thresholds[2])[0]
    bounding_boxes = bounding_boxes[keep]
    bounding_boxes[:, 4] = probs[keep, 1].reshape((-1,))
    offsets = offsets[keep]
    landmarks = landmarks[keep]

    bounding_boxes = calibrate_box(bounding_boxes, offsets)
    keep = nms(bounding_boxes, nms_thresholds[2], mode='min')
    
    featureOutputs = featureOutputs[keep]

    return featureOutputs

In [5]:
dataset_path = '../Dataset/emotiw/'

processed_dataset_path = '../Dataset/FaceFeatures/'

In [6]:
image_datasets = {x : datasets.ImageFolder(os.path.join(dataset_path, x))
                    for x in ['train', 'val']}

class_names = image_datasets['train'].classes

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
class_names

['Negative', 'Neutral', 'Positive']

In [8]:
training_dataset = image_datasets['train']
validation_dataset = image_datasets['val']

In [9]:
neg_train = sorted(os.listdir(dataset_path + 'train/Negative/'))
neu_train = sorted(os.listdir(dataset_path + 'train/Neutral/'))
pos_train = sorted(os.listdir(dataset_path + 'train/Positive/'))

neg_val = sorted(os.listdir(dataset_path + 'val/Negative/'))
neu_val = sorted(os.listdir(dataset_path + 'val/Neutral/'))
pos_val = sorted(os.listdir(dataset_path + 'val/Positive/'))

In [10]:
neg_train_filelist = [x.split('.')[0] for x in neg_train]
neu_train_filelist = [x.split('.')[0] for x in neu_train]
pos_train_filelist = [x.split('.')[0] for x in pos_train]

neg_val_filelist = [x.split('.')[0] for x in neg_val]
neu_val_filelist = [x.split('.')[0] for x in neu_val]
pos_val_filelist = [x.split('.')[0] for x in pos_val]


In [11]:
neg_train_filelist = neg_train_filelist[1:]

neg_val_filelist = neg_val_filelist[1:]
neu_val_filelist = neu_val_filelist[1:]
pos_val_filelist = pos_val_filelist[1:]

In [30]:
print(neg_train_filelist[:10])
print(neu_train_filelist[:10])
print(pos_train_filelist[:10])

print(neg_val_filelist[:10])
print(neu_val_filelist[:10])
print(pos_val_filelist[:10])

['neg_1', 'neg_10', 'neg_100', 'neg_1000', 'neg_1001', 'neg_1002', 'neg_1003', 'neg_1004', 'neg_1005', 'neg_1006']
['neu_1', 'neu_10', 'neu_100', 'neu_1000', 'neu_1001', 'neu_1002', 'neu_1003', 'neu_1004', 'neu_1005', 'neu_1006']
['pos_1', 'pos_10', 'pos_100', 'pos_1000', 'pos_1001', 'pos_1002', 'pos_1003', 'pos_1004', 'pos_1005', 'pos_1006']
['neg_1', 'neg_10', 'neg_100', 'neg_1000', 'neg_1001', 'neg_1002', 'neg_1003', 'neg_1004', 'neg_1005', 'neg_1006']
['neu_1', 'neu_10', 'neu_100', 'neu_1000', 'neu_1001', 'neu_1002', 'neu_1003', 'neu_1004', 'neu_1005', 'neu_1006']
['pos_1', 'pos_10', 'pos_100', 'pos_1000', 'pos_1001', 'pos_1002', 'pos_1003', 'pos_1004', 'pos_1005', 'pos_1006']


In [31]:
train_filelist = neg_train_filelist + neu_train_filelist + pos_train_filelist
val_filelist = neg_val_filelist + neu_val_filelist + pos_val_filelist

In [32]:
print(len(training_dataset))
print(len(validation_dataset))


9815
4346


In [23]:
with open('hello.text', 'a') as f:
    f.write(train_filelist[i])

In [28]:
# for i in range(len(training_dataset)):
# # for i in range(1):
#     image, label = training_dataset[i]
#     print(train_filelist[i])
#     try:
#         if label == 0:
#             if os.path.isfile(processed_dataset_path + 'train/Negative/' + train_filelist[i] + '.npz'):
#                 print(train_filelist[i] + ' Already present')
#                 continue
#             features = get_face_features(image)
#             if(type(features)) == tuple:
#                 with open('hello.text', 'a') as f:
#                     f.write(train_filelist[i])
#                 continue
#             features = features.data.numpy()
# #             print(features)
# #             print(features.shape)
#             if features.size == 0:
#                 print('MTCNN model handling empty face condition at ' + train_filelist[i])
#             np.savez(processed_dataset_path + 'train/Negative/' + train_filelist[i] , a=features)
#         elif label == 1:
#             if os.path.isfile(processed_dataset_path + 'train/Neutral/' + train_filelist[i] + '.npz'):
#                 print(train_filelist[i] + ' Already present')
#                 continue
#             features = get_face_features(image)
#             if(type(features)) == tuple:
#                 with open('hello.text', 'a') as f:
#                     f.write(train_filelist[i])
#                 continue
#             features = features.data.numpy()
# #             print(features.shape)
#             if features.size == 0:
#                 print('MTCNN model handling empty face condition at ' + train_filelist[i])
#             np.savez(processed_dataset_path + 'train/Neutral/' + train_filelist[i] , a=features)
#         else:
#             if os.path.isfile(processed_dataset_path + 'train/Positive/' + train_filelist[i] + '.npz'):
#                 print(train_filelist[i] + ' Already present')
#                 continue
#             features = get_face_features(image)
#             if(type(features)) == tuple:
#                 with open('hello.text', 'a') as f:
#                     f.write(train_filelist[i])
#                 continue
#             features = features.data.numpy()
# #             print(features.shape)
#             if features.size == 0:
#                 print('MTCNN model handling empty face condition at ' + train_filelist[i])
#             np.savez(processed_dataset_path + 'train/Positive/' + train_filelist[i] , a=features)            
            
#     except ValueError:
#         print('No faces detected for ' + train_filelist[i] + ". Also MTCNN failed.")
#         if label == 0:
#             np.savez(processed_dataset_path + 'train/Negative/' + train_filelist[i] , a=np.zeros(1))
#         elif label == 1:
#             np.savez(processed_dataset_path + 'train/Neutral/' + train_filelist[i] , a=np.zeros(1))
#         else:
#             np.savez(processed_dataset_path + 'train/Positive/' + train_filelist[i] , a=np.zeros(1))
#         continue

neg_1
neg_1 Already present
neg_10
neg_10 Already present
neg_100
neg_100 Already present
neg_1000
neg_1000 Already present
neg_1001
neg_1001 Already present
neg_1002
neg_1002 Already present
neg_1003
neg_1003 Already present
neg_1004
neg_1004 Already present
neg_1005
neg_1005 Already present
neg_1006
neg_1006 Already present
neg_1007
neg_1007 Already present
neg_1008
neg_1008 Already present
neg_1009
neg_1009 Already present
neg_101
neg_101 Already present
neg_1010
neg_1010 Already present
neg_1011
neg_1011 Already present
neg_1012
neg_1012 Already present
neg_1013
neg_1013 Already present
neg_1014
neg_1014 Already present
neg_1015
neg_1015 Already present
neg_1016
neg_1016 Already present
neg_1017
neg_1017 Already present
neg_1018
neg_1018 Already present
neg_1019
neg_1019 Already present
neg_102
neg_102 Already present
neg_1020
neg_1020 Already present
neg_1021
neg_1021 Already present
neg_1022
neg_1022 Already present
neg_1023
neg_1023 Already present
neg_1024
neg_1024 Already pres

neg_1224 Already present
neg_1225
neg_1225 Already present
neg_1226
neg_1226 Already present
neg_1227
neg_1227 Already present
neg_1228
neg_1228 Already present
neg_1229
neg_1229 Already present
neg_123
neg_123 Already present
neg_1230
neg_1230 Already present
neg_1231
neg_1231 Already present
neg_1232
neg_1232 Already present
neg_1233
neg_1233 Already present
neg_1234
neg_1234 Already present
neg_1235
neg_1235 Already present
neg_1236
neg_1236 Already present
neg_1237
neg_1237 Already present
neg_1238
neg_1238 Already present
neg_1239
neg_1239 Already present
neg_124
neg_124 Already present
neg_1240
neg_1240 Already present
neg_1241
neg_1241 Already present
neg_1242
neg_1242 Already present
neg_1243
neg_1243 Already present
neg_1244
neg_1244 Already present
neg_1245
neg_1245 Already present
neg_1246
neg_1246 Already present
neg_1247
neg_1247 Already present
neg_1248
neg_1248 Already present
neg_1249
neg_1249 Already present
neg_125
neg_125 Already present
neg_1250
neg_1250 Already pre

neg_1443 Already present
neg_1444
neg_1444 Already present
neg_1445
neg_1445 Already present
neg_1446
neg_1446 Already present
neg_1447
neg_1447 Already present
neg_1448
neg_1448 Already present
neg_1449
neg_1449 Already present
neg_145
neg_145 Already present
neg_1450
neg_1450 Already present
neg_1451
neg_1451 Already present
neg_1452
neg_1452 Already present
neg_1453
neg_1453 Already present
neg_1454
neg_1454 Already present
neg_1455
neg_1455 Already present
neg_1456
neg_1456 Already present
neg_1457
neg_1457 Already present
neg_1458
neg_1458 Already present
neg_1459
neg_1459 Already present
neg_146
neg_146 Already present
neg_1460
neg_1460 Already present
neg_1461
neg_1461 Already present
neg_1462
neg_1462 Already present
neg_1463
neg_1463 Already present
neg_1464
neg_1464 Already present
neg_1465
neg_1465 Already present
neg_1466
neg_1466 Already present
neg_1467
neg_1467 Already present
neg_1468
neg_1468 Already present
neg_1469
neg_1469 Already present
neg_147
neg_147 Already pre

neg_1665
neg_1665 Already present
neg_1666
neg_1666 Already present
neg_1667
neg_1667 Already present
neg_1668
neg_1668 Already present
neg_1669
neg_1669 Already present
neg_167
neg_167 Already present
neg_1670
neg_1670 Already present
neg_1671
neg_1671 Already present
neg_1672
neg_1672 Already present
neg_1673
neg_1673 Already present
neg_1674
neg_1674 Already present
neg_1675
neg_1675 Already present
neg_1676
neg_1676 Already present
neg_1677
neg_1677 Already present
neg_1678
neg_1678 Already present
neg_1679
neg_1679 Already present
neg_168
neg_168 Already present
neg_1680
neg_1680 Already present
neg_1681
neg_1681 Already present
neg_1682
neg_1682 Already present
neg_1683
neg_1683 Already present
neg_1684
neg_1684 Already present
neg_1685
neg_1685 Already present
neg_1686
neg_1686 Already present
neg_1687
neg_1687 Already present
neg_1688
neg_1688 Already present
neg_1689
neg_1689 Already present
neg_169
neg_169 Already present
neg_1690
neg_1690 Already present
neg_1691
neg_1691 Al

neg_1888 Already present
neg_1889
neg_1889 Already present
neg_189
neg_189 Already present
neg_1890
neg_1890 Already present
neg_1891
neg_1891 Already present
neg_1892
neg_1892 Already present
neg_1893
neg_1893 Already present
neg_1894
neg_1894 Already present
neg_1895
neg_1895 Already present
neg_1896
neg_1896 Already present
neg_1897
neg_1897 Already present
neg_1898
neg_1898 Already present
neg_1899
neg_1899 Already present
neg_19
neg_19 Already present
neg_190
neg_190 Already present
neg_1900
neg_1900 Already present
neg_1901
neg_1901 Already present
neg_1902
neg_1902 Already present
neg_1903
neg_1903 Already present
neg_1904
neg_1904 Already present
neg_1905
neg_1905 Already present
neg_1906
neg_1906 Already present
neg_1907
neg_1907 Already present
neg_1908
neg_1908 Already present
neg_1909
neg_1909 Already present
neg_191
neg_191 Already present
neg_1910
neg_1910 Already present
neg_1911
neg_1911 Already present
neg_1912
neg_1912 Already present
neg_1913
neg_1913 Already present

...

pos_1254
pos_1255
pos_1256
pos_1257
pos_1258
pos_1259
pos_126
pos_1260
pos_1261
pos_1262
pos_1263
pos_1264
pos_1265
pos_1266
pos_1267
pos_1268
pos_1269
pos_127
pos_1270
pos_1271
pos_1272
pos_1273
pos_1274
pos_1275
pos_1276
pos_1277
pos_1278
pos_1279
pos_128
pos_1280
pos_1281
pos_1282
pos_1283
pos_1284
pos_1285
pos_1286
pos_1287
pos_1288
pos_1289
pos_129
pos_1290
pos_1291
pos_1292
pos_1293
pos_1294
pos_1295
pos_1296
pos_1297
pos_1298
pos_1299
pos_13
pos_130
pos_1300
pos_1301
pos_1302
pos_1303
pos_1304
pos_1305
pos_1306
pos_1307
pos_1308
pos_1309
pos_131
pos_1310
pos_1311
pos_1312
pos_1313
pos_1314
pos_1315
pos_1316
pos_1317
pos_1318
pos_1319
pos_132
pos_1320
pos_1321
pos_1322
pos_1323
pos_1324
pos_1325
pos_1326
pos_1327
pos_1328
pos_1329
pos_133
pos_1330
pos_1331
pos_1332
pos_1333
pos_1334
pos_1335
pos_1336
pos_1337
pos_1338
pos_1339
pos_134
pos_1340
pos_1341
pos_1342
pos_1343
pos_1344
pos_1345
pos_1346
pos_1347
pos_1348
pos_1349
pos_135
pos_1350
pos_1351
pos_1352
pos_1353
pos_1354
pos_

pos_2057
pos_2058
pos_2059
pos_206
pos_2060
pos_2061
pos_2062
pos_2063
pos_2064
pos_2065
pos_2066
pos_2067
pos_2068
pos_2069
pos_207
pos_2070
pos_2071
pos_2072
pos_2073
pos_2074
pos_2075
pos_2076
pos_2077
pos_2078
pos_2079
pos_208
pos_2080
pos_2081
pos_2082
pos_2083
pos_2084
pos_2085
pos_2086
pos_2087
pos_2088
pos_2089
pos_209
pos_2090
pos_2091
pos_2092
pos_2093
pos_2094
pos_2095
pos_2096
pos_2097
pos_2098
pos_2099
pos_21
pos_210
pos_2100
pos_2101
pos_2102
pos_2103
pos_2104
pos_2105
pos_2106
pos_2107
pos_2108
pos_2109
pos_211
pos_2110
pos_2111
pos_2112
pos_2113
pos_2114
pos_2115
pos_2116
pos_2117
pos_2118
pos_2119
pos_212
pos_2120
pos_2121
pos_2122
pos_2123
pos_2124
pos_2125
pos_2126
pos_2127
pos_2128
pos_2129
pos_213
pos_2130
pos_2131
pos_2132
pos_2133
pos_2134
MTCNN model handling empty face condition at pos_2134
pos_2135
pos_2136
pos_2137
pos_2138
pos_2139
pos_214
pos_2140
pos_2141
pos_2142
pos_2143
pos_2144
pos_2145
pos_2146
pos_2147
pos_2148
pos_2149
pos_215
pos_2150
pos_2151
pos_

pos_286
pos_2860
pos_2861
pos_2862
pos_2863
pos_2864
pos_2865
pos_2866
pos_2867
pos_2868
pos_2869
pos_287
pos_2870
pos_2871
pos_2872
pos_2873
pos_2874
pos_2875
pos_2876
pos_2877
pos_2878
pos_2879
pos_288
pos_2880
pos_2881
pos_2882
pos_2883
pos_2884
pos_2885
pos_2886
pos_2887
pos_2888
pos_2889
pos_289
pos_2890
pos_2891
pos_2892
pos_2893
pos_2894
pos_2895
pos_2896
pos_2897
pos_2898
pos_2899
pos_29
pos_290
pos_2900
pos_2901
pos_2902
pos_2903
pos_2904
pos_2905
pos_2906
pos_2907
pos_2908
pos_2909
pos_291
pos_2910
pos_2911
pos_2912
pos_2913
pos_2914
pos_2915
pos_2916
pos_2917
pos_2918
pos_2919
pos_292
pos_2920
pos_2921
pos_2922
pos_2923
pos_2924
pos_2925
pos_2926
pos_2927
pos_2928
pos_2929
pos_293
pos_2930
pos_2931
pos_2932
pos_2933
pos_2934
pos_2935
pos_2936
pos_2937
pos_2938
pos_2939
pos_294
pos_2940
pos_2941
pos_2942
pos_2943
pos_2944
pos_2945
pos_2946
pos_2947
pos_2948
pos_2949
pos_295
pos_2950
pos_2951
pos_2952
pos_2953
pos_2954
pos_2955
pos_2956
pos_2957
pos_2958
pos_2959
pos_296
pos_2

pos_3662
pos_3663
pos_3664
pos_3665
pos_3666
pos_3667
pos_3668
pos_3669
pos_367
pos_3670
pos_3671
pos_3672
pos_3673
pos_3674
pos_3675
pos_3676
pos_3677
pos_3678
pos_3679
pos_368
pos_3680
pos_3681
pos_3682
pos_3683
pos_3684
pos_3685
pos_3686
pos_3687
pos_3688
pos_3689
pos_369
pos_3690
pos_3691
pos_3692
pos_3693
pos_3694
pos_3695
pos_3696
pos_3697
pos_3698
pos_3699
pos_37
pos_370
pos_3700
pos_3701
pos_3702
pos_3703
pos_3704
pos_3705
pos_3706
pos_3707
pos_3708
pos_3709
pos_371
pos_3710
pos_3711
pos_3712
pos_3713
pos_3714
pos_3715
pos_3716
pos_3717
pos_3718
pos_3719
pos_372
pos_3720
pos_3721
pos_3722
pos_3723
pos_3724
pos_3725
pos_3726
pos_3727
pos_3728
pos_3729
pos_373
pos_3730
pos_3731
pos_3732
pos_3733
pos_3734
pos_3735
pos_3736
pos_3737
pos_3738
pos_3739
pos_374
pos_3740
pos_3741
pos_3742
pos_3743
pos_3744
pos_3745
pos_3746
pos_3747
pos_3748
pos_3749
pos_375
pos_3750
pos_3751
pos_3752
pos_3753
pos_3754
pos_3755
pos_3756
pos_3757
pos_3758
pos_3759
pos_376
pos_3760
pos_3761
pos_3762
pos_

pos_966
pos_967
pos_968
pos_969
pos_97
pos_970
pos_971
pos_972
pos_973
pos_974
pos_975
pos_976
pos_977
pos_978
pos_979
pos_98
pos_980
pos_981
pos_982
pos_983
pos_984
pos_985
pos_986
pos_987
pos_988
pos_989
pos_99
pos_990
pos_991
pos_992
pos_993
pos_994
pos_995
pos_996
pos_997
pos_998
pos_999


In [34]:
for i in range(len(validation_dataset)):
# for i in range(1):
    image, label = validation_dataset[i]
    print(val_filelist[i])
    try:
        if label == 0:
            if os.path.isfile(processed_dataset_path + 'val/Negative/' + val_filelist[i] + '.npz'):
                print(val_filelist[i] + ' Already present')
                continue
            features = get_face_features(image)
            if(type(features)) == tuple:
                with open('hello.text', 'a') as f:
                    f.write(val_filelist[i])
                continue
            features = features.data.numpy()
#             print(features)
#             print(features.shape)
            if features.size == 0:
                print('MTCNN model handling empty face condition at ' + val_filelist[i])
            np.savez(processed_dataset_path + 'val/Negative/' + val_filelist[i] , a=features)
        elif label == 1:
            if os.path.isfile(processed_dataset_path + 'val/Neutral/' + val_filelist[i] + '.npz'):
                print(val_filelist[i] + ' Already present')
                continue
            features = get_face_features(image)
            if(type(features)) == tuple:
                with open('hello.text', 'a') as f:
                    f.write(val_filelist[i])
                continue
            features = features.data.numpy()
#             print(features.shape)
            if features.size == 0:
                print('MTCNN model handling empty face condition at ' + val_filelist[i])
            np.savez(processed_dataset_path + 'val/Neutral/' + val_filelist[i] , a=features)
        else:
            if os.path.isfile(processed_dataset_path + 'val/Positive/' + val_filelist[i] + '.npz'):
                print(val_filelist[i] + ' Already present')
                continue
            features = get_face_features(image)
            if(type(features)) == tuple:
                with open('hello.text', 'a') as f:
                    f.write(val_filelist[i])
                continue
            features = features.data.numpy()
#             print(features.shape)
            if features.size == 0:
                print('MTCNN model handling empty face condition at ' + val_filelist[i])
            np.savez(processed_dataset_path + 'val/Positive/' + val_filelist[i] , a=features)            
            
    except ValueError:
        print('No faces detected for ' + val_filelist[i] + ". Also MTCNN failed.")
        if label == 0:
            np.savez(processed_dataset_path + 'val/Negative/' + val_filelist[i] , a=np.zeros(1))
        elif label == 1:
            np.savez(processed_dataset_path + 'val/Neutral/' + val_filelist[i] , a=np.zeros(1))
        else:
            np.savez(processed_dataset_path + 'val/Positive/' + val_filelist[i] , a=np.zeros(1))
        continue

neg_1
neg_10
neg_100
MTCNN model handling empty face condition at neg_100
neg_1000
neg_1001
neg_1002
neg_1003
neg_1004
neg_1005
neg_1006
neg_1007
neg_1008
neg_1009
MTCNN model handling empty face condition at neg_1009
neg_101
neg_1010
neg_1011
neg_1012
neg_1013
neg_1014
MTCNN model handling empty face condition at neg_1014
neg_1015
neg_1016
neg_1017
neg_1018
neg_1019
neg_102
neg_1020
neg_1021
neg_1022
neg_1023
neg_1024
neg_1025
neg_1026
neg_1027
neg_1028
neg_1029
neg_103
neg_1030
neg_1031
neg_1032
neg_1033
neg_1034
neg_1035
neg_1036
neg_1037
neg_1038
neg_1039
neg_104
neg_1040
neg_1041
neg_1042
neg_1043
neg_1044
neg_1045
neg_1046
neg_1047
neg_1048
neg_1049
neg_105
neg_1050
neg_1051
neg_1052
neg_1053
neg_1054
neg_1055
neg_1056
neg_1057
neg_1058
neg_1059
neg_106
neg_1060
neg_1061
neg_1062
neg_1063
neg_1064
neg_1065
neg_1066
neg_1067
neg_1068
neg_1069
neg_107
neg_1070
neg_1071
neg_1072
neg_1073
neg_1074
neg_1075
neg_1076
neg_1077
neg_1078
neg_1079
neg_108
neg_1080
neg_1081
neg_1082
neg_108

neg_754
neg_755
neg_756
neg_757
neg_758
MTCNN model handling empty face condition at neg_758
neg_759
neg_76
neg_760
neg_761
neg_762
neg_763
neg_764
neg_765
neg_766
neg_767
neg_768
neg_769
neg_77
neg_770
neg_771
neg_772
neg_773
neg_774
neg_775
neg_776
MTCNN model handling empty face condition at neg_776
neg_777
neg_778
neg_779
neg_78
neg_780
neg_781
neg_782
neg_783
neg_784
neg_785
neg_786
neg_787
neg_788
neg_789
neg_79
neg_790
neg_791
neg_792
neg_793
neg_794
neg_795
neg_796
neg_797
neg_798
neg_799
neg_8
neg_80
neg_800
neg_801
neg_802
neg_803
neg_804
neg_805
neg_806
neg_807
neg_808
neg_809
neg_81
neg_810
neg_811
neg_812
neg_813
neg_814
neg_815
neg_816
neg_817
neg_818
neg_819
neg_82
neg_820
neg_821
neg_822
neg_823
neg_824
neg_825
neg_826
neg_827
neg_828
neg_829
neg_83
neg_830
neg_831
neg_832
neg_833
neg_834
neg_835
neg_836
neg_837
neg_838
neg_839
neg_84
neg_840
neg_841
neg_842
MTCNN model handling empty face condition at neg_842
neg_843
neg_844
neg_845
neg_846
neg_847
neg_848
neg_849
neg_

neu_307
neu_308
neu_309
neu_31
neu_310
neu_311
neu_312
neu_313
neu_314
neu_315
neu_316
neu_317
neu_318
neu_319
neu_32
neu_320
neu_321
neu_322
neu_323
neu_324
neu_325
neu_326
neu_327
neu_328
neu_329
neu_33
neu_330
neu_331
neu_332
neu_333
neu_334
neu_335
neu_336
neu_337
neu_338
neu_339
neu_34
neu_340
neu_341
neu_342
neu_343
neu_344
neu_345
neu_346
neu_347
neu_348
neu_349
neu_35
neu_350
neu_351
neu_352
neu_353
neu_354
neu_355
neu_356
neu_357
neu_358
neu_359
MTCNN model handling empty face condition at neu_359
neu_36
neu_360
neu_361
neu_362
neu_363
neu_364
neu_365
neu_366
neu_367
neu_368
neu_369
neu_37
neu_370
neu_371
neu_372
neu_373
neu_374
neu_375
neu_376
neu_377
neu_378
neu_379
neu_38
neu_380
neu_381
neu_382
neu_383
neu_384
neu_385
neu_386
neu_387
neu_388
neu_389
neu_39
neu_390
neu_391
neu_392
neu_393
neu_394
neu_395
neu_396
neu_397
neu_398
MTCNN model handling empty face condition at neu_398
neu_399
neu_4
neu_40
neu_400
neu_401
neu_402
neu_403
neu_404
neu_405
neu_406
neu_407
neu_408
ne

pos_1183
pos_1184
pos_1185
pos_1186
pos_1187
pos_1188
pos_1189
pos_119
pos_1190
pos_1191
pos_1192
pos_1193
pos_1194
pos_1195
pos_1196
pos_1197
pos_1198
pos_1199
pos_12
pos_120
pos_1200
pos_1201
pos_1202
pos_1203
pos_1204
pos_1205
pos_1206
pos_1207
pos_1208
pos_1209
pos_121
pos_1210
pos_1211
pos_1212
pos_1213
pos_1214
pos_1215
pos_1216
pos_1217
pos_1218
pos_1219
pos_122
pos_1220
pos_1221
pos_1222
pos_1223
pos_1224
pos_1225
pos_1226
pos_1227
pos_1228
pos_1229
pos_123
pos_1230
pos_1231
pos_1232
pos_1233
pos_1234
pos_1235
pos_1236
pos_1237
pos_1238
pos_1239
pos_124
pos_1240
pos_1241
pos_1242
pos_1243
pos_1244
pos_1245
pos_1246
pos_1247
pos_1248
pos_1249
pos_125
pos_1250
pos_1251
pos_1252
pos_1253
pos_1254
pos_1255
pos_1256
pos_1257
pos_1258
pos_1259
pos_126
pos_1260
pos_1261
pos_1262
pos_1263
pos_1264
pos_1265
pos_1266
pos_1267
pos_1268
pos_1269
pos_127
pos_1270
pos_1271
pos_1272
pos_1273
pos_1274
pos_1275
pos_1276
pos_1277
pos_1278
pos_1279
pos_128
pos_1280
pos_1281
pos_1282
pos_1283
pos_

pos_438
pos_439
pos_44
pos_440
pos_441
pos_442
pos_443
pos_444
pos_445
pos_446
pos_447
pos_448
pos_449
pos_45
pos_450
pos_451
pos_452
pos_453
pos_454
pos_455
pos_456
pos_457
pos_458
pos_459
pos_46
pos_460
pos_461
pos_462
pos_463
pos_464
pos_465
pos_466
pos_467
pos_468
pos_469
pos_47
pos_470
pos_471
pos_472
pos_473
pos_474
pos_475
pos_476
pos_477
pos_478
pos_479
pos_48
pos_480
pos_481
pos_482
pos_483
pos_484
pos_485
pos_486
pos_487
pos_488
pos_489
pos_49
pos_490
pos_491
pos_492
pos_493
pos_494
pos_495
pos_496
pos_497
pos_498
pos_499
pos_5
pos_50
pos_500
pos_501
pos_502
pos_503
pos_504
pos_505
pos_506
pos_507
pos_508
pos_509
pos_51
pos_510
pos_511
pos_512
pos_513
pos_514
pos_515
pos_516
pos_517
pos_518
pos_519
pos_52
pos_520
pos_521
pos_522
pos_523
pos_524
pos_525
pos_526
pos_527
pos_528
pos_529
pos_53
pos_530
pos_531
pos_532
pos_533
pos_534
pos_535
pos_536
pos_537
pos_538
pos_539
pos_54
pos_540
pos_541
pos_542
pos_543
pos_544
pos_545
pos_546
pos_547
pos_548
pos_549
pos_55
pos_550
pos_55